In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
BASE_URL = 'https://www.basketball-reference.com/'

draft_2012 = 'https://www.basketball-reference.com/draft/NBA_2012.html'
draft_2013 = 'https://www.basketball-reference.com/draft/NBA_2013.html'
draft_2014 = 'https://www.basketball-reference.com/draft/NBA_2014.html'
draft_2015 = 'https://www.basketball-reference.com/draft/NBA_2015.html'
draft_2016 = 'https://www.basketball-reference.com/draft/NBA_2016.html'
draft_2017 = 'https://www.basketball-reference.com/draft/NBA_2017.html'
draft_2018 = 'https://www.basketball-reference.com/draft/NBA_2018.html'

draft_2021 = 'https://www.basketball-reference.com/draft/NBA_2021.html'

list_url = [ draft_2012, draft_2013, draft_2014, 
            draft_2015, draft_2016, draft_2017, draft_2018]
# list_url = [draft_2021]

div class="table_container is_setup" id="div_stats"
    table class="sortable stats_table now_sortable" id="stats" data-cols-to-freeze="1,2,3"
        tbody
            tr data-row="0" class="rowSum"
                td data-stat="pick_overall"      => ***PICK RANK***
                td data-stat="player"
                    a href="***URL STATS NBA***" => ***NOM***
                td data-stat="college_name"
                    a                            => ***College name***

In [3]:
players = []

for url in list_url:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find('div',class_="table_container").find('table', class_='sortable stats_table').find('tbody')

    for player in table.find_all('tr'):
        college = None #initiate college variable as not every player comes from an american university
        player_name = None
        for td in player.find_all('td'):
            if td['data-stat'] == "player":
                player_url = BASE_URL+td.find('a').get('href')
                player_name = td.find('a').string
            elif td['data-stat'] == "pick_overall":
                pick_rank = td.find('a').string
            elif td['data-stat'] == "college_name" and td.find('a'):
                college = td.find('a').string
        if player_name:
            players.append({'player': player_name, 'pick rank':pick_rank , 'year': url[47:51], 'college':college, 'url': player_url})


In [4]:
players[0]['url']

'https://www.basketball-reference.com//players/d/davisan02.html'

In [5]:
def convert_height(feet,inches):
    h_ft = int(feet)
    h_inch = int(inches)

    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54, 1)
    return h_cm

In [8]:
for player in players:
    player_response = requests.get(player['url'])
    player_soup = BeautifulSoup(player_response.content, "html.parser")
    
    feet, inches = player_soup.find('span', itemprop="height").string.split('-')
    player['height (cm)']=convert_height(feet,inches)
    
    player['weight (lb)']=player_soup.find('span', itemprop="weight").string[0:3]
    
    for li in player_soup.find_all('li'):
        if li.find('a') and li.find('a').string=='College Basketball at Sports-Reference.com':
            uni_url_string = li.find('a').get('href')
            player_id = uni_url_string.split('/')[-1][:-5]
            player['uni_url']=uni_url_string
            player['player_id']=player_id
            break

In [9]:
players_df = pd.DataFrame(players)
players_df

,player,pick rank,year,college,url,height (cm),weight (lb),uni_url,player_id
0,Anthony Davis,1,2012,Kentucky,https://www.basketball-reference.com//players/...,208.3,253,https://www.sports-reference.com/cbb/players/a...,anthony-davis-5
1,Michael Kidd-Gilchrist,2,2012,Kentucky,https://www.basketball-reference.com//players/...,198.1,232,https://www.sports-reference.com/cbb/players/m...,michael-kidd-gilchrist-1
2,Bradley Beal,3,2012,Florida,https://www.basketball-reference.com//players/...,190.5,207,https://www.sports-reference.com/cbb/players/b...,bradley-beal-1
3,Dion Waiters,4,2012,Syracuse,https://www.basketball-reference.com//players/...,190.5,215,https://www.sports-reference.com/cbb/players/d...,dion-waiters-1
4,Thomas Robinson,5,2012,Kansas,https://www.basketball-reference.com//players/...,208.3,237,https://www.sports-reference.com/cbb/players/t...,thomas-robinson-2
...,...,...,...,...,...,...,...,...,...
415,Ray Spalding,56,2018,Louisville,https://www.basketball-reference.com//players/...,208.3,215,https://www.sports-reference.com/cbb/players/r...,raymond-spalding-1
416,Kevin Hervey,57,2018,UT Arlington,https://www.basketball-reference.com//players/...,205.7,230,https://www.sports-reference.com/cbb/players/k...,kevin-hervey-1
417,Thomas Welsh,58,2018,UCLA,https://www.basketball-reference.com//players/...,213.4,255,https://www.sports-reference.com/cbb/players/t...,thomas-welsh-1
418,George King,59,2018,Colorado,https://www.basketball-reference.com//players/...,198.1,225,https://www.sports-reference.com/cbb/players/g...,george-king-1


In [10]:
players_df.to_csv('../raw_data/players.csv', index=False)

In [ ]:
# players_df.to_csv('../raw_data/draft_2021.csv', index=False)